In [ ]:
# Load required libraries
library(shiny)
library(ggplot2)
library(caret)
library(e1071)
library(dplyr)
install.packages("kernlab")


# Load Iris dataset
data(iris)

In [ ]:


# Define UI
ui <- fluidPage(
  titlePanel("Iris Flower Classification"),
  sidebarLayout(
    sidebarPanel(
      selectInput("feature1", "Select Feature 1:", choices = colnames(iris)[1:4]),
      selectInput("feature2", "Select Feature 2:", choices = colnames(iris)[1:4]),
      actionButton("train_btn", "Train Model")
    ),
    mainPanel(
      plotOutput("scatter_plot"),
      verbatimTextOutput("model_summary")
    )
  )
)

# Define server logic
server <- function(input, output) {
  
  # Reactive expression for training model
  trained_model <- eventReactive(input$train_btn, {
    features <- c(input$feature1, input$feature2)
    formula <- as.formula(paste("Species ~", paste(features, collapse = "+")))
    model <- train(formula, data = iris, method = "svmRadial")
    return(model)
  })
  
  # Render scatter plot
  output$scatter_plot <- renderPlot({
    model <- trained_model()
    features <- c(input$feature1, input$feature2)
    ggplot(iris, aes_string(x = features[1], y = features[2], color = "Species")) +
      geom_point() +
      geom_point(data = model$finalModel$SV, color = "black", size = 3, shape = 1) +
      theme_minimal() +
      labs(title = "Iris Flower Classification")
  })
  
  # Display model summary
  output$model_summary <- renderPrint({
    model <- trained_model()
    summary(model)
  })
}

# Run the application
shinyApp(ui = ui, server = server)
